<a href="https://colab.research.google.com/github/MahammadKhalid/Enhancing-Product-Design-through-AI-Driven-Sentiment-Analysis-of-Amazon-Reviews-using-BERT/blob/main/BERT_Classifier_Tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers -U
!pip install torch
!pip install accelerate==0.20.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 4.9 MB/s eta 0:00:00


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification
import pandas as pd
import numpy as np

In [ ]:
# Replace 'your_file.xlsx' with the path to your Excel file
file_path = '/content/drive/MyDrive/NLP/Amazon Reviews Validation Data.xlsx'

# Read the Excel file into a Pandas DataFrame
try:
    Amazon_Reviews = pd.read_excel(file_path)
    # Print the first 5 rows of the DataFrame for verification
except FileNotFoundError:
    print(f"File '{file_path}' not found.")
except pd.errors.ParserError:
    print(f"Error occurred while parsing the Excel file '{file_path}'. Please check the file format and structure.")
except Exception as e:
    print(f"An error occurred: {str(e)}")

In [ ]:
Amazon_Reviews.head(2)

,Reviews,Aspect
0,The settings are perfect for all hair types,Adaptability
1,with lots of settings,Adaptability


Assiging Copy to Emotion_Training_Data

In [ ]:
Reviews_Training=pd.read_csv('/content/drive/MyDrive/NLP/cheAmazon Reviews Validation Data.xlsx')
Reviews_Training=Amazon_Reviews.drop_duplicates(subset=['Reviews'])
Amazon_Reviews_validation = Amazon_Reviews.drop_duplicates(subset=['Reviews'])
Amazon_Reviews.dropna(inplace=True)
Reviews_Training.dropna(inplace=True)

<ipython-input-8-bbee1a459cae>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Reviews_Training.dropna(inplace=True)


In [ ]:
emotion_training_data=Reviews_Training
emotion_validation_data=Amazon_Reviews_validation

In [ ]:
emotion_training_data = emotion_training_data.rename(columns={'Reviews': 'Comment_Text', 'Aspect': 'Label'})
emotion_validation_data = emotion_validation_data.rename(columns={'Reviews': 'Comment_Text', 'Aspect': 'Label'})

In [ ]:
emotion_training_data['Label'].unique()

array(['Adaptability', 'Durability', 'Ease of Use', 'Ergonomics',
       'Interference', 'Performance', 'Use Efficiency', 'Aesthetics',
       'Ease of Reprocessing', 'Ease of Storage', 'Price', 'Safety'],
      dtype=object)

In [ ]:
original_labels = [
    'Adaptability', 'Durability', 'Ease of Use', 'Ergonomics',
    'Interference', 'Performance', 'Use Efficiency', 'Aesthetics',
    'Ease of Reprocessing', 'Ease of Storage', 'Price', 'Safety'
]

# Map labels to numerical values (0 to 12)
labeling_dict = {label: idx for idx, label in enumerate(original_labels)}

In [ ]:
emotion_training_data['Label']=emotion_training_data['Label'].map(labeling_dict)
emotion_validation_data['Label']=emotion_validation_data['Label'].map(labeling_dict)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=12)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
X_train =emotion_training_data["Comment_Text"].to_list()
X_val=emotion_validation_data["Comment_Text"].to_list()
y_train = emotion_training_data["Label"].to_list()
y_val=emotion_validation_data["Label"].to_list()

In [ ]:
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)


In [ ]:
X_train_tokenized.keys()


dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:

# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [ ]:
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [ ]:
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    return {"accuracy": accuracy}

In [ ]:
# Define Trainer
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=10,
    per_device_train_batch_size=50

)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)



In [ ]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=220, training_loss=0.6295355363325639, metrics={'train_runtime': 410.6031, 'train_samples_per_second': 26.595, 'train_steps_per_second': 0.536, 'total_flos': 1363757146315200.0, 'train_loss': 0.6295355363325639, 'epoch': 10.0})

In [ ]:
Model_validation_metrics=trainer.evaluate()

In [ ]:
Loss=Model_validation_metrics['eval_loss']
Accuracy=Model_validation_metrics['eval_accuracy']

In [ ]:
print(f'Validation_Loss is : {Loss}, and Validation_Accuracy is : {Accuracy}')

Validation_Loss is : 0.044630296528339386, and Validation_Accuracy is : 0.9981684981684982


In [ ]:
trainer.save_model('/content/drive/MyDrive/NLP/CustomModel')
tokenizer.save_pretrained("/content/drive/MyDrive/NLP/Tokenizer")

('Tokenizer/tokenizer_config.json',
 'Tokenizer/special_tokens_map.json',
 'Tokenizer/vocab.txt',
 'Tokenizer/added_tokens.json')

In [ ]:
tokenizer = BertTokenizer.from_pretrained('Tokenizer')
model = BertForSequenceClassification.from_pretrained("CustomModel")

In [ ]:
encoded_input = tokenizer("Input for single value for multiple we will run for loop", return_tensors='pt',padding=True,truncation=True)
output = model(**encoded_input)
logits = output.logits.detach().cpu().numpy()
y_pred = np.argmax(logits,axis=-1)
given_value =y_pred[0]
# Get the key for the given value
result_key = next(key for key, value in labeling_dict.items() if value == given_value)
# Print the result
print(result_key)